# The Decorator Pattern

This pattern allows you to attach additional responsibilities to an object at runtime without modifying it.

Sometimes, to add new functionalities to our system we end up implementing an unnecessary, complex class hierarchy that leads to the 'class explosion' problem. For example, in the context of a videogame, we can have a base enemy class, but we also want to be able to add extras to it such as helmets or armors. If we define one class for each possible combination, we will end up getting too many classes: one for the BaseEnemy, and some more for the BaseEnemyWithHelmet, BaseEnemyWithArmor and BaseEnemyWithArmorAndHelmet. That's too many classes! And, of course, if we want to add a new extra such as gloves, we will have to create even more classes. This is a case of 'class explosion' and it's not the better way to go.

The Decorator Pattern can help us with this. But first, let's define the terminology:
- **Component Interface**: An interface for the base objects.
- **Component**: The object that may be decorated. In the previous example, this would be the 'EnemyBase' class. Every *Component* of the system implements the *Component Interface*.
- **Decorator**: The extras that will be added to the base *Component* We will have one *Decorator* for each 'extra'. 

Using this pattern, we can add all the *Decorators* that we want to our base *Component*. It doesn't matter if we want to add just one *Decorator* or multiple of them: the pattern works for any number. It means that we can decorate the base class or decorated classes too. It's like an onion: we can add all the layers that we want. But keep in mind that sometimes the order of these layers does matter.

If you have a base *Component* and a decorated *Component*, you should be able to use them interchangeably. So, the *Decorator* classes must inherit from the *Component* class too, so *Decorator* **is a** *Component* as well. Simultaneously, the *Decorator* class must **have a** *Component* inside it - that is, a composition. In the previous example, we could have an *armor Decorator* that has a *helmet Decorator* that has the *base enemy Component*. This recursive architecture is the thing that allows the decorator pattern to be implemented. 

Note as well that all the messages received by the most superficial *Decorator* can be transferred over the layers to reach the base *Component*; Those layers can modify that message too, or add new behaviours.

Here is the UML diagram of the Decorator Pattern:

![](UML-Diagrams/Decorator-ClassDiagram.png)

And here is the implementation of it:

In [1]:
from abc import ABC, abstractmethod

# Let's define the abstract class for the Component
class Component(ABC):
    @abstractmethod
    def __init__(self) -> None:
        """Constructor."""

    @abstractmethod
    def operation(self) -> None:
        """An operation."""


# And now let's inherit a concrete component
class ConcreteComponent(Component):
    def __init__(self) -> None:
        # here you may have some attribs...
        pass

    def operation(self) -> None:
        print("Do something")


# Let's create the abstract class for the Decorators
class Decorator(Component):  # inheritance: a Decorator *is a* component
    @abstractmethod
    def __init__(self, component: Component) -> None:
        """Constructor."""


# And let's implement for example 2 concrete Decorators
class ConcreteDecorator1(Decorator):
    def __init__(self, component: Component) -> None:
        self._component = (
            component  # composition: a Decorator *has a* component
        )

    def operation(self) -> None:
        print("Do something different!")


class ConcreteDecorator2(Decorator):
    def __init__(self, component: Component) -> None:
        self._component = component

    def operation(self) -> None:
        print("Do something new!")


if __name__ == "__main__":

    # First, we create our base component
    base_component = ConcreteComponent()
    base_component.operation()  # this component has a default behaviour

    # Then we add our first decorator. We inject the base component
    # as an input parameter
    decorated_component = ConcreteDecorator1(base_component)
    decorated_component.operation()  # the decorated component has a different behaviour

    # Finally we add another decorator.
    decoratedx2_component = ConcreteDecorator2(decorated_component)
    decoratedx2_component.operation()  # a new behaviour

    # We could even continue adding more decorators...
    decoratedx3_component = ConcreteDecorator2(decoratedx2_component)
    decoratedx4_component = ConcreteDecorator1(decoratedx3_component)
    # ... & more


Do something
Do something different!
Do something new!


## Example: 

You have an app that sends messages by email and you want to extend it to notify some info by multiple platforms: Facebook, SMS; Whatsapp, Slack... and so on. You want to be able to notify to one **or more** of these, so in order to avoid having 1 class per combination, you decide to implement it with a Decorator Pattern.

In [2]:
from abc import ABC, abstractmethod


# Let's create our component
class Notifier(ABC):
    @abstractmethod
    def send_notification(self, user: str, message: str) -> None:
        """Sends a notification to a user."""


class EmailNotifier(Notifier):
    def send_notification(self, user: dict, message: str) -> None:
        """Sends a notification by email to a user."""

        try:
            user_email = user["email"]
        except KeyError as e:
            print(f"Unknown Email. Cannot send message: {message}.")
        else:
            print(f"--> Notifying user {user_email} by email: {message}")


# Let's create the abstract class of a decorator
class NotifierDecorator(Notifier):
    @abstractmethod
    def __init__(self, notifier: Notifier) -> None:
        """Constructor"""


# And now let's create a decorator to send notifications
# to Facebook too
class FacebookNotifier(NotifierDecorator):
    def __init__(self, notifier: Notifier) -> None:
        self._notifier = notifier

    def send_notification(self, user: dict, message: str) -> None:
        """Further sends a notification by Facebook to a user."""

        try:
            user_facebookID = user["FacebookID"]
        except KeyError as e:
            print(f"Unknown Facebook ID. Cannot send message: {message}.")
        else:
            print(
                f"--> Notifying user {user_facebookID} by Facebook: {message}"
            )
        finally:
            # We can propagate info over layers as well
            self._notifier.send_notification(user, message)


# And now let's create a decorator to send notifications
# to Whatsapp too
class WhatsappNotifier(NotifierDecorator):
    def __init__(self, notifier: Notifier) -> None:
        self._notifier = notifier

    def send_notification(self, user: dict, message: str) -> None:
        """Further sends a notification by Whatsapp to every registered user."""

        try:
            user_phone = user["phone"]
        except KeyError as e:
            print(f"Unknown phone number. Cannot send message: {message}.")
        else:
            print(f"--> Notifying user {user_phone} by Whatsapp: {message}")
        finally:
            self._notifier.send_notification(user, message)


if __name__ == "__main__":

    # Let's create a notifier for email, Facebook and Whatsapp
    email_notifier = EmailNotifier()
    facebook_notifier = FacebookNotifier(email_notifier)
    whatsapp_facebook_notifier = WhatsappNotifier(facebook_notifier)

    # Let's suppose these are the users of our platform
    users_info = {
        "User1": {
            "email": "user1@gmail.com",
            "FacebookID": "9024592",
            "phone": "696101028",
        },
        "User2": {
            "email": "user2@live.com",
            # "FacebookID": "2034022",  # Maybe some info is missing
            "phone": "696440253",
        },
    }

    # Let's notify them
    message = "Hi! I'm using the Decorator Pattern!"
    for user, user_info in users_info.items():
        print(f"Notifying user {user}...")
        whatsapp_facebook_notifier.send_notification(user_info, message)


Notifying user User1...
--> Notifying user 696101028 by Whatsapp: Hi! I'm using the Decorator Pattern!
--> Notifying user 9024592 by Facebook: Hi! I'm using the Decorator Pattern!
--> Notifying user user1@gmail.com by email: Hi! I'm using the Decorator Pattern!
Notifying user User2...
--> Notifying user 696440253 by Whatsapp: Hi! I'm using the Decorator Pattern!
Unknown Facebook ID. Cannot send message: Hi! I'm using the Decorator Pattern!.
--> Notifying user user2@live.com by email: Hi! I'm using the Decorator Pattern!
